## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-02-10-45-09 +0000,wapo,Government shutdown continues with no end in s...,https://www.washingtonpost.com/business/2025/1...
1,2025-10-02-10-44-42 +0000,bbc,Tesco warns 'enough is enough' on business taxes,https://www.bbc.com/news/articles/c784q26v57vo...
2,2025-10-02-10-44-23 +0000,nyt,Philippines President Says Tents Needed to She...,https://www.nytimes.com/2025/10/02/world/asia/...
3,2025-10-02-10-43-43 +0000,nypost,Amazon unveils ‘price-conscious’ grocery brand...,https://nypost.com/2025/10/02/business/amazon-...
4,2025-10-02-10-43-42 +0000,wapo,Russian gasoline production buckles under Ukra...,https://www.washingtonpost.com/world/2025/10/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2374/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,shutdown,39
195,trump,33
0,government,32
120,new,18
671,will,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
148,2025-10-01-22-43-00 +0000,wsj,Many GOP senators seeking an end to the govern...,https://www.wsj.com/politics/policy/white-hous...,146
156,2025-10-01-22-10-12 +0000,nypost,Trump admin will keep Statue of Liberty open d...,https://nypost.com/2025/10/01/us-news/trump-ad...,134
322,2025-10-01-13-50-00 +0000,wsj,The White House and lawmakers failed to reach ...,https://www.wsj.com/politics/policy/government...,116
134,2025-10-02-00-00-00 +0000,wsj,Office of Management and Budget Director Russe...,https://www.wsj.com/politics/policy/russell-vo...,114
207,2025-10-01-20-13-16 +0000,wapo,Trump administration seeks to exert control ov...,https://www.washingtonpost.com/business/2025/1...,112


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
148,146,2025-10-01-22-43-00 +0000,wsj,Many GOP senators seeking an end to the govern...,https://www.wsj.com/politics/policy/white-hous...
293,102,2025-10-01-15-14-50 +0000,nypost,Supreme Court sets date for Lisa Cook’s mortga...,https://nypost.com/2025/10/01/us-news/supreme-...
277,67,2025-10-01-16-38-26 +0000,bbc,Israel 'tightens siege' of Gaza City as Hamas ...,https://www.bbc.com/news/articles/cvgnznkjrx6o...
139,52,2025-10-01-23-09-00 +0000,wsj,The Trump administration said it is withholdin...,https://www.wsj.com/politics/policy/trump-admi...
251,48,2025-10-01-18-02-32 +0000,bbc,Jimmy Kimmel on the moment he found out he was...,https://www.bbc.com/news/videos/c87yql1ll8po?a...
316,45,2025-10-01-14-05-00 +0000,wsj,White House nominees to lead the BLS and the C...,https://www.wsj.com/politics/policy/white-hous...
46,40,2025-10-02-08-58-26 +0000,nypost,Portland police seek public’s help after ‘safe...,https://nypost.com/2025/10/02/us-news/portland...
67,38,2025-10-02-05-15-26 +0000,nypost,Trump admin to provide Ukraine with intelligen...,https://nypost.com/2025/10/02/us-news/trump-ad...
109,34,2025-10-02-01-33-00 +0000,wsj,Rescuers Race to Free Boys Trapped in Collapse...,https://www.wsj.com/world/asia/rescuers-race-t...
138,34,2025-10-01-23-14-00 +0000,wsj,France Detains Russian ‘Shadow Fleet’ Tanker S...,https://www.wsj.com/world/europe/france-detain...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
